Import Dependencies

In [1]:
import pandas as pd
import os
import datetime as dt
import spmfunctions.spmfunctions as spm
import warnings
warnings.filterwarnings('ignore')

In [2]:
intx='SH-44-Hartley'
f_pkl='df_raw.pk'
yymm=''
sfx=''
sfx_sum='_hourly'

df_mvmt,df_exc,df_adv,df_sb=spm.det_cfg('./{}/Configuration/'.format(intx))
ph_dets=[(int(i[1]),int(j.loc['Det 1'])) for i,j in df_sb.iterrows()]

path_pkl = '.\\{}\df_raw'.format(intx)

df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))

df_res=pd.DataFrame()

for ph, det in ph_dets:
    print('Starting Ph {} {}'.format(ph, dt.datetime.now()))
    df_comb=spm.comb_gyr_det(df_raw)
    df_aog=spm.arrival_on_green(df_comb, [det], ph, tt=0, det_code=82)
    df_res=df_res.append(df_aog)
    print('Ph {} Complete {}'.format(ph, dt.datetime.now()))

if not os.path.exists('./{}/AOG'.format(intx)):
    os.mkdir('./{}/AOG'.format(intx))

df_res.index.name='Cycle_start'
df_c=df_comb.loc[:,['Cycle_start','Coord_plan']].drop_duplicates()
df_res=pd.merge(df_res,df_c,'left', on='Cycle_start').fillna(0)
spm.bin_AOG(df_res).to_csv('./{}/AOG/{}_aog{}{}.csv'.format(intx,intx,yymm,sfx_sum), index=False)
df_res.to_csv('./{}/AOG/{}_aog{}{}.csv'.format(intx,intx,yymm,sfx),index=False)

Starting Ph 1 2023-12-07 10:49:01.226902
Ph 1 Complete 2023-12-07 10:49:02.109747
Starting Ph 2 2023-12-07 10:49:02.109747
Ph 2 Complete 2023-12-07 10:49:02.969211
Starting Ph 3 2023-12-07 10:49:02.969211
Ph 3 Complete 2023-12-07 10:49:03.824912
Starting Ph 4 2023-12-07 10:49:03.824912
Ph 4 Complete 2023-12-07 10:49:04.740104
Starting Ph 5 2023-12-07 10:49:04.740104
Ph 5 Complete 2023-12-07 10:49:05.609777
Starting Ph 6 2023-12-07 10:49:05.609777
Ph 6 Complete 2023-12-07 10:49:06.536987
Starting Ph 7 2023-12-07 10:49:06.536987
Ph 7 Complete 2023-12-07 10:49:07.402077
Starting Ph 8 2023-12-07 10:49:07.402077
Ph 8 Complete 2023-12-07 10:49:08.285842


NameError: name 'resdf' is not defined

In [4]:
sfx='_single_adv'
sfx_sum='_single_adv_hourly'

det=2

intersections=[#'Chinden',
               #'Colchester',
               #'Fairview','Franklin',
               #'HobbleCreek',
               'I84WB','IslandWoods',#,'McMillan',
               'Pine','RiverValley',
               'StLuke','Ustick',
               'Riverside','Wainwright','I84EB']#Village

months='all' # list of months YYMM Can use 'all' to process all df_raw files in df_raw folder

tt=0
det_code=82 #81=deactivate, 82=activate

for intx in intersections:
    print('starting {} {}'.format(intx,str(dt.datetime.now())))
    # Get phase, detector pairs from dets.csv configuration file
    
    try:
        df_mvmt,df_exc,df_adv,df_sb=spm.det_cfg('./ACHD_Eagle-{}/Configuration/'.format(intx))
        ph_dt=[(int(i[1]),int(j.loc['Det {}'.format(det)])) for i,j in df_adv.iterrows()]
    except:
        print('Error processing {}'.format(intx))
        continue
    
    path_pkl = '.\\ACHD_Eagle-{}\\df_raw'.format(intx)    
    
    if months=='all':
        pkls=os.listdir(path_pkl)
    else:
        pkls=['df_raw_{}.pk'.format(m) for m in months]    
    
    for f_pkl in pkls:
        yymm=f_pkl[-7:-3]
        
        # Read df_raw from pickle and generate df_comb
        df_raw=pd.read_pickle(os.path.join(path_pkl,f_pkl))
        print('starting {} {}'.format(f_pkl,str(dt.datetime.now())))
        df_comb=spm.comb_gyr_det(df_raw)

        # Initiate results dataframe
        resdf=pd.DataFrame()
        for p, d in ph_dt:
            print('starting Ph{} {}'.format(p,str(dt.datetime.now())))
            df_aog=spm.arrival_on_green(df_comb, [d], p, tt=tt, det_code=det_code)
            resdf=resdf.append(df_aog)
            #AOG=df_aog.AoG.sum()/df_aog.Total.sum()
            #print('phase%s: %s'%(p, AOG))

        # Format and write results dataframe to file
        resdf.index.name='Cycle_start'
        df_c=df_comb.loc[:,['Cycle_start','Coord_plan']].drop_duplicates()
        resdf=pd.merge(resdf,df_c,'left', on='Cycle_start').fillna(0)
        
        if not os.path.exists('./ACHD_Eagle-{}/AOG'.format(intx)):
            os.mkdir('./ACHD_Eagle-{}/AOG'.format(intx))
        
        spm.bin_AOG(resdf).to_csv('./ACHD_Eagle-{}/AOG/{}_aog{}{}.csv'.format(intx,intx,yymm,sfx_sum), index=False)
        resdf.to_csv('./ACHD_Eagle-{}/AOG/{}_aog{}{}.csv'.format(intx,intx,yymm,sfx),index=False)
    print('finished {} {}'.format(f_pkl,str(dt.datetime.now())))

starting I84WB 2023-12-06 22:46:29.496400
Error processing I84WB
starting IslandWoods 2023-12-06 22:46:29.514343
starting df_raw_2304.pk 2023-12-06 22:46:29.970460
starting Ph4 2023-12-06 22:46:40.268349
starting Ph8 2023-12-06 22:46:42.371684
starting df_raw_2305.pk 2023-12-06 22:46:45.432261
starting Ph4 2023-12-06 22:46:56.831495
starting Ph8 2023-12-06 22:46:58.963167
starting df_raw_2306.pk 2023-12-06 22:47:02.125461
starting Ph4 2023-12-06 22:47:14.098935
starting Ph8 2023-12-06 22:47:16.486449
starting df_raw_2307.pk 2023-12-06 22:47:19.336523
starting Ph4 2023-12-06 22:47:30.306600
starting Ph8 2023-12-06 22:47:32.402852
starting df_raw_2308.pk 2023-12-06 22:47:35.303869
starting Ph4 2023-12-06 22:47:46.275934
starting Ph8 2023-12-06 22:47:48.377315
starting df_raw_2309.pk 2023-12-06 22:47:51.246103
starting Ph4 2023-12-06 22:48:01.835437
starting Ph8 2023-12-06 22:48:03.914850
starting df_raw_2310.pk 2023-12-06 22:48:06.818901
starting Ph4 2023-12-06 22:48:18.005396
starting P

In [16]:
for intx in ['Chinden','Colchester','Fairview','Franklin',
             'HobbleCreek','I84WB','IslandWoods','McMillan',
             'Pine','RiverValley','StLuke','Ustick',
             'Riverside','Village','Wainwright','I84EB']:
    df_mvmt,df_exc,df_adv,df_sb=spm.det_cfg('./ACHD_Eagle-%s/Configuration/'%intx)
    print(intx)
    for df in [df_adv,df_sb]:
        ph_dt=[(int(i[1]),int(j.loc['Det 1'])) for i,j in df.iterrows()]
        print(ph_dt)

Chinden
[(2, 54), (4, 62), (6, 50), (8, 58)]
[(1, 1), (2, 6), (3, 9), (4, 15), (5, 5), (6, 2), (7, 13), (8, 10)]
Colchester
[(2, 54), (4, 62), (6, 50), (8, 58)]
[(1, 1), (2, 6), (3, 9), (4, 14), (5, 5), (6, 2), (7, 13), (8, 10)]
Fairview
[(2, 54), (4, 62), (6, 50), (8, 58)]
[]
Franklin
[(2, 54), (4, 62), (6, 50), (8, 58)]
[(1, 1), (2, 6), (3, 9), (4, 15), (5, 5), (6, 2), (7, 13), (8, 10)]
HobbleCreek
[(4, 62), (8, 58)]
[(2, 5), (3, 9), (4, 14), (6, 1), (7, 13), (8, 10)]
I84WB
[(2, 54), (4, 61), (8, 57)]
[]
IslandWoods
[(4, 62), (8, 58)]
[(2, 5), (3, 9), (4, 14), (6, 1), (7, 13), (8, 10)]
McMillan
[(2, 53), (4, 61), (6, 49), (8, 57)]
[(1, 1), (2, 6), (3, 9), (4, 14), (5, 5), (6, 2), (7, 13), (8, 10)]
Pine
[(2, 54), (4, 62), (6, 50), (8, 58)]
[]
RiverValley
[(4, 61), (8, 57)]
[(1, 1), (2, 6), (3, 9), (4, 15), (5, 5), (6, 2), (7, 13), (8, 10)]
StLuke
[(4, 62), (8, 58)]
[]
Ustick
[(2, 54), (4, 62), (6, 50), (8, 58)]
[(1, 1), (2, 6), (3, 9), (4, 14), (5, 5), (6, 2), (7, 13), (8, 10)]
Rivers